In [1]:
from dqn.dqn_base import ConcreteDQN
from dqn.dqn_memory import DQNWithMemory
from dqn.dqn_targetnn import DQNWithTargetNet
from dqn.neural_net import NeuralNetworkWithCNN
from dqn.memory import ExperienceReplay
import gymnasium as gym
import torch
import random
import matplotlib.pyplot as plt
plt.style.use('_mpl-gallery')

In [18]:
def preprocess_state(state, device):
    state = state.transpose((2, 0, 1))
    state = torch.from_numpy(state)
    state = state.to(device, dtype=torch.float32)
    state = state.unsqueeze(1)
    return state

In [19]:
device = torch.device("cpu")

env_id = "PongNoFrameskip-v4"

env = gym.make(env_id)

seed_value = 23

torch.manual_seed(seed_value)
random.seed(seed_value)

learning_rate = 0.005
num_episodes = 500
gamma = 0.9999

hidden_layer = 512

report_interval = 20
number_of_inputs = env.observation_space.shape[0]
number_of_outputs = env.action_space.n


In [20]:
qnet_agent = ConcreteDQN(
    env,
    NeuralNetworkWithCNN(number_of_inputs, hidden_layer, number_of_outputs).to(device),
    learning_rate,
    device,
    gamma,
    preprocess_state_func=preprocess_state
)

qnet_agent_with_memory = DQNWithMemory(
    qnet_agent,
    ExperienceReplay(50000),
    32,
)

qnet_agent_with_targetnn = DQNWithTargetNet(
    qnet_agent,
    NeuralNetworkWithCNN(number_of_inputs, hidden_layer, number_of_outputs).to(device),
)

qnet_agent_with_memory_and_targetnn = DQNWithTargetNet(
    qnet_agent_with_memory,
    NeuralNetworkWithCNN(number_of_inputs, hidden_layer, number_of_outputs).to(device),
)

qnet_agent_with_memory_and_dueling = DQNWithTargetNet(
    qnet_agent_with_memory,
    NeuralNetworkWithCNN(number_of_inputs, hidden_layer, number_of_outputs).to(
        device
    ),
)

In [21]:
qnet_agent.learn(500, seed_value, report_interval)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x22528 and 3136x512)